In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import pprint
from itertools import combinations
import pickle
import config.config as conf

/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/ipykernel_39012/1527706816.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select \
                  x.q_id \
                , x.q_posttypeid \
                , x.q_acceptedanswerid \
                , x.q_parentid \
                , x.q_creationdate \
                , x.q_score \
                , x.q_viewcount \
                , x.q_owneruserid \
                , x.q_title \
                , x.q_tags \
                , x.q_answercount \
                , x.q_commentcount \
                , x.q_reputation \
                , b.id               as a_id \
                , b.posttypeid       as a_posttypeid \
                , b.acceptedanswerid as a_acceptedanswerid \
                , b.parentid         as a_parentid \
                , b.creationdate     as a_creationdate \
                , b.score            as a_score \
                , b.viewcount        as a_viewcount \
                , b.owneruserid      as a_owneruserid \
                , b.title            as a_title \
                , b.tags             as a_tags \
                , b.answercount      as a_answercount \
                , b.commentcount     as a_commentcount \
                , d.reputation       as a_reputation \
  from ( \
           select a.id               as q_id \
                , a.posttypeid       as q_posttypeid \
                , a.acceptedanswerid as q_acceptedanswerid \
                , a.parentid         as q_parentid \
                , a.creationdate     as q_creationdate \
                , a.score            as q_score \
                , a.viewcount        as q_viewcount \
                , a.owneruserid      as q_owneruserid \
                , a.title            as q_title \
                , a.answercount      as q_answercount \
                , a.commentcount     as q_commentcount \
                , c.reputation       as q_reputation \
                , replace(replace(lower(a.tags), '<', ''), '>', ' ')as q_tags \
           from posts a \
              , users c \
           where a.creationdate >= '2023-08-03' \
             and a.posttypeid = '1' \
             and a.owneruserid is not null \
             and c.id = a.owneruserid \
             and (replace(replace(lower(a.tags), '<', ''), '>', ' ') like '%python %') \
       )   x \
        , posts b \
        , users d \
where b.parentid = x.q_id \
  and b.posttypeid = '2' \
  and b.owneruserid is not null \
  and d.id = b.owneruserid \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [4]:
df = pd.DataFrame(rows, columns = [
  'q_id' 
, 'q_posttypeid' 
, 'q_acceptedanswerid'
, 'q_parentid' 
, 'q_creationdate' 
, 'q_score' 
, 'q_viewcount' 
, 'q_owneruserid' 
, 'q_title' 
, 'q_tags' 
, 'q_answercount' 
, 'q_commentcount' 
, 'q_reputation' 
, 'a_id' 
, 'a_posttypeid' 
, 'a_acceptedanswerid' 
, 'a_parentid' 
, 'a_creationdate' 
, 'a_score' 
, 'a_viewcount' 
, 'a_owneruserid' 
, 'a_title' 
, 'a_tags' 
, 'a_answercount'
, 'a_commentcount'
, 'a_reputation' 
])

In [5]:
df.head()

,q_id,q_posttypeid,q_acceptedanswerid,q_parentid,q_creationdate,q_score,q_viewcount,q_owneruserid,q_title,q_tags,...,a_parentid,a_creationdate,a_score,a_viewcount,a_owneruserid,a_title,a_tags,a_answercount,a_commentcount,a_reputation
0,77578447,1,77578902.0,None,2023-11-30 12:31:21.097,0,62,23014409,Printing to table form from a list and a neste...,python dictionary for-loop iteration,...,77578447,2023-11-30 12:54:20.073,0,None,2079189,None,None,None,0,4298
1,77495792,1,NaN,None,2023-11-16 14:45:26.583,0,24,10244869,Cronjob not running inside docker container,python docker cron dockerfile,...,77495792,2023-11-30 12:56:46.023,0,None,6482931,None,None,None,0,1571
2,77578194,1,77578772.0,None,2023-11-30 11:46:58.733,1,52,22951744,How to allocate list of product quantities con...,python numpy,...,77578194,2023-11-30 13:00:15.673,0,None,5168011,None,None,None,0,47897
3,77578661,1,NaN,None,2023-11-30 13:02:52.423,0,49,3120467,Python-Selenium-Xpath - Web table - Get Span t...,python html selenium-webdriver xpath,...,77578661,2023-11-30 15:02:46.523,1,None,23015287,None,None,None,0,11
4,77579021,1,77579081.0,None,2023-11-30 13:55:13.467,1,42,21440005,Variable assignment difference,python for-loop while-loop variable-assignment...,...,77579021,2023-11-30 14:02:52.353,2,None,2422776,None,None,None,0,299523


In [6]:
print(df['q_creationdate'].min())
print(df['a_creationdate'].max())

2023-08-03 00:07:37.917000
2023-12-03 10:11:57.450000


In [7]:
df_gephi = df[['q_owneruserid', 'a_owneruserid']]

In [8]:
df_gephi.columns = ['Source', 'Target']

In [9]:
df_gephi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29219 entries, 0 to 29218
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Source  29219 non-null  int64
 1   Target  29219 non-null  int64
dtypes: int64(2)
memory usage: 456.7 KB


In [10]:
np.append(df_gephi['Source'].values, df_gephi['Target'].values)

array([23014409, 10244869, 22951744, ..., 18590799, 22961148,    37213])

In [11]:
node = np.unique(np.append(df_gephi['Source'].values, df_gephi['Target'].values))

In [12]:
# 질문자->답변자 로 edgelist 생성
edgelist = list(zip(df_gephi['Source'], df_gephi['Target']))

In [13]:
df_gephi.to_csv('../../data/python_gephi.csv', index=False)

In [14]:
print(df['q_owneruserid'].min())
print(df['q_owneruserid'].max())

116
23031413


In [15]:
df_question = df_gephi['Source'].value_counts().reset_index()
df_answer = df_gephi['Target'].value_counts().reset_index()

In [16]:
df_gephi

,Source,Target
0,23014409,2079189
1,10244869,6482931
2,22951744,5168011
3,3120467,23015287
4,21440005,2422776
...,...,...
29214,22190754,22190754
29215,23014233,22687704
29216,22842584,18590799
29217,10574250,22961148


In [17]:
df_answer = df_answer.rename(columns = {'Target': 'user_id', 'count':'answer_cnt'})
df_question = df_question.rename(columns = {'Source': 'user_id', 'count':'question_cnt'})

In [18]:
df_qna = pd.merge(df_question, df_answer, on = 'user_id', how='outer')

In [19]:
df_qna.fillna(0, inplace=True)

In [20]:
df_qna

,user_id,question_cnt,answer_cnt
0,116,2.0,0.0
1,476,0.0,11.0
2,1527,0.0,4.0
3,2988,0.0,2.0
4,3063,0.0,1.0
...,...,...,...
24134,23030687,1.0,1.0
24135,23030938,1.0,0.0
24136,23031306,0.0,1.0
24137,23031346,1.0,0.0


In [21]:
# user별 question/answer count를 가지고 있는 데이터
with open('../../data/python_df_qna.pkl', 'wb') as f:
    pickle.dump(df_qna, f)


In [22]:
# gephi 시각화용 데이터 
with open('../../data/python_df_gephi.pkl', 'wb') as f:
    pickle.dump(df_gephi, f)

In [23]:
# 전체 데이터셋 
with open('../../data/python_df.pkl', 'wb') as f:
    pickle.dump(df, f)